In [1]:
import torch
import torch.nn as nn
from torch.nn import Linear
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch.utils.data import DataLoader,Dataset,TensorDataset
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import numpy as np
from scipy.sparse import coo_matrix
import pandas as pd
import tqdm
import random
import datetime
import sys
from scipy.stats import pearsonr
# from  torch.utils.data import TensorDataset,DataLoader

## 参数

In [2]:
TIME_LEN = 12
BATCH_SIZE = 32

In [3]:
torch.manual_seed(12345)
torch.cuda.manual_seed(12345)
np.random.seed(12345)
random.seed(12345)
torch.backends.cudnn.deterministic = True

### 加载数据

In [4]:
data = np.load('/home/hqh/DataSetFile/metrla/STdata.npy')#.astype(np.float16)# 时空
print(data.shape)

(34272, 207)


In [5]:
A = np.load('/home/hqh/DataSetFile/metrla/adj01.npy')
adj = coo_matrix(A)
values = adj.data  
indices = np.vstack((adj.row, adj.col))  # 我们真正需要的coo形式
edge_index  = torch.LongTensor(indices)  # PyG框架需要的coo形式
print(edge_index)

tensor([[  0,   0,   0,  ..., 206, 206, 206],
        [  0,  13,  37,  ..., 187, 198, 206]])


In [6]:
n_V,n_E = len(A),len(edge_index[0])
print(n_E,n_V)

1722 207


### 同配异配

In [7]:
# start_time = datetime.datetime.now()
# DATA_ARRAY =  np.repeat(np.expand_dims(data,2),len(adj),2) # time,n,n,每一行的元素一样
# print(datetime.datetime.now()-start_time)
# DIFF = DATA_ARRAY - DATA_ARRAY.transpose(0,2,1) #time,n,n， i,j表示d当前时间步的data[i]-data[j]
# print(datetime.datetime.now()-start_time)
# DIFF = DIFF/DATA_ARRAY
# print(datetime.datetime.now()-start_time)

In [8]:
start_time = datetime.datetime.now()
DIFF = data[:,edge_index[0]]-data[:,edge_index[1]]
print(datetime.datetime.now()-start_time)
DIFF = (DIFF+1)/(data[:,edge_index[0]]+1)
print(datetime.datetime.now()-start_time)
DIFF_ABS = np.absolute(DIFF)
print(datetime.datetime.now()-start_time)

0:00:01.090931
0:00:02.302211
0:00:02.626245


In [9]:
eplision=0.02
print(sys.getsizeof(DIFF) / 1024 / 1024/1024, 'GB')
print(DIFF.shape)
number = DIFF.shape[0]*DIFF.shape[1]
# print( len(DIFF[np.isnan(DIFF)])/number)
# print( len(DIFF[np.isinf(DIFF)])/number)
print(DIFF_ABS.mean(),DIFF_ABS.std())
print('同配比例：',((DIFF_ABS<=eplision).sum())/number)
print('异配正比例：',((DIFF>eplision).sum())/number)
print('异配负比例：',((DIFF<-eplision).sum())/number)

0.4397064447402954 GB
(34272, 1722)
0.9486729411882304 6.329240266700626
同配比例： 0.2273149097037189
异配正比例： 0.4731735173744294
异配负比例： 0.29951157292185165


In [10]:
DIFF[DIFF>eplision]=1
DIFF[DIFF_ABS<=eplision]=0.5
DIFF[DIFF<-eplision]=-1

In [11]:
print(DIFF.shape)
print((DIFF==1).sum()/number,(DIFF==-1).sum()/number)
print(DIFF[0][10:188])

(34272, 1722)
0.4731735173744294 0.29951157292185165
[-1.   0.5  0.5  1.   1.   1.   1.   1.   1.   0.5  0.5  0.5  0.5  1.
  1.   0.5  1.   1.   0.5  0.5 -1.  -1.  -1.   0.5 -1.  -1.   1.  -1.
 -1.   0.5 -1.   1.  -1.   1.   0.5  1.   0.5  1.   1.   0.5 -1.   1.
  1.   1.   0.5  1.   1.   0.5  0.5  1.   1.   0.5  1.   1.   1.   0.5
  1.   1.   1.   1.   1.   0.5  0.5  1.   1.   0.5  1.   0.5  1.   0.5
  1.   1.   0.5  0.5  1.   0.5  0.5  1.   0.5  1.   1.   1.   0.5  0.5
  0.5  1.   0.5  1.  -1.  -1.   0.5  0.5  0.5 -1.  -1.   0.5 -1.  -1.
  1.  -1.   0.5  1.   1.   0.5  1.   1.   1.   1.   1.  -1.   0.5 -1.
  1.  -1.  -1.   1.  -1.  -1.  -1.   1.  -1.   1.  -1.   1.   0.5  1.
  1.   1.   1.   1.   1.   1.   1.   0.5  0.5  1.   0.5 -1.  -1.  -1.
  1.   1.   0.5  1.   1.   1.   1.   1.   0.5  1.  -1.  -1.   1.  -1.
 -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
 -1.  -1.   0.5  0.5 -1.   0.5  0.5 -1.  -1.  -1. ]


### 变化方向

### 1.直接求出sign

In [12]:
EXPAND_DATA = data[:,edge_index[0]]
sign = EXPAND_DATA[1:]-EXPAND_DATA[:-1]
sign = (sign+1)/(EXPAND_DATA[:-1]+1)
sign_abs = np.absolute(sign)
print(sign.shape)

(34271, 1722)


In [13]:
number = sign.shape[0]*sign.shape[1]
print(sign_abs.mean(),sign_abs.std())
print('时序同配比例：',((sign_abs<=eplision).sum())/number)
print('时序异配同向比例：',((sign>eplision).sum())/number)
print('时序异配负向比例：',((sign<-eplision).sum())/number)

0.3516844111856842 3.646938386908529
时序同配比例： 0.33384646005428276
时序异配同向比例： 0.49182567545671957
时序异配负向比例： 0.17432786448899767


In [14]:
import matplotlib.pyplot as plt
def show_month(sign,sign_abs,DIFF,DIFF_ABS,eplision):
    number = sign.shape[1]
    T_homo = (sign_abs<=eplision).sum(-1)/number
    T_hete1 = (sign>eplision).sum(-1)/number
    T_hete2 = (sign<-eplision).sum(-1)/number
    
    

In [15]:
# sign[sign_abs<=eplision]=0
# sign[sign>eplision]=1
# sign[sign<-eplision]=-1

In [16]:
# print(sign.shape)

### 2.但是直接求sign会有标签暴露之嫌疑，还是需要用一个网络来学习sign

### 2.1加载数据

In [17]:
sig = data[1:]-data[0:-1]
sig = (sig+1)/(data[0:-1]+1)
sig_abs = np.absolute(sig)
print(sig.shape)

(34271, 207)


In [18]:
eplision=0.02
number = sig.shape[0]*sig.shape[1]
print(sig_abs.mean(),sig_abs.std())
print('时序同配比例：',((sig_abs<=eplision).sum())/number)
print('时序异配同向比例：',((sig>eplision).sum())/number)
print('时序异配负向比例：',((sig<-eplision).sum())/number)

0.35385622143903406 3.6673369425688302
时序同配比例： 0.3325436627099968
时序异配同向比例： 0.4919509840364461
时序异配负向比例： 0.17550535325355715


In [19]:
sig[sig_abs<=eplision]=0
sig[sig>eplision]=1
sig[sig<-eplision]=2

In [20]:
print(sig.shape)

(34271, 207)


In [21]:
x = []
for i in range(1,len(data)-TIME_LEN): #从第2个开始，和sig保持一致
    x.append(data[i:i+TIME_LEN])
x = torch.FloatTensor(x).transpose(2,1)#np的transpose和torch的不一样？
print(x.shape)# batch  空间 时间

sig_x = []
y = []
for i in range(0,len(sig)-TIME_LEN):
    sig_x.append(sig[i:i+TIME_LEN])
    y.append(sig[i+TIME_LEN])
sig_x = torch.FloatTensor(sig_x).transpose(2,1)
y = torch.LongTensor(y)
print(sig_x.shape)

input_x = torch.cat((x,sig_x),dim=-1)
print('y.shape:',y.shape)
print('input_x.shape:',input_x.shape)

torch.Size([34259, 207, 12])
torch.Size([34259, 207, 12])
y.shape: torch.Size([34259, 207])
input_x.shape: torch.Size([34259, 207, 24])


/tmp/ipykernel_25543/1059340229.py:13: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  y = torch.LongTensor(y)


In [22]:
print(y.dtype,y[0])

torch.int64 tensor([2, 2, 1, 2, 1, 2, 1, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 2, 1, 2, 2, 2, 2,
        1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 0, 1, 0, 1, 2, 0,
        1, 2, 2, 2, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 0, 0, 2, 0,
        2, 1, 0, 1, 1, 2, 1, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0,
        0, 1, 2, 1, 0, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
        2, 1, 2, 0, 0, 2, 0, 0, 1, 1, 2, 2, 1, 1, 0, 1, 1, 1, 0, 2, 0, 2, 1, 1,
        2, 1, 2, 2, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 2, 1, 1, 0, 0, 0, 0, 2, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 2, 1, 1, 2, 1, 1, 0, 2, 0])


In [23]:
BATCH_SIZE=128
def construct_dataloader(x,y,loader_type):
    dataset = TensorDataset(x,y)
    if loader_type=='train':
        dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)# 不能shuffle
    else:
        dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)# 
    return dataloader

In [24]:
num_samples = len(input_x)
train_ratio = 0.8
test_ratio = 0.1

num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train
x_train, y_train = input_x[:num_train], y[:num_train]
x_val, y_val =  input_x[num_train: num_train + num_val],y[num_train: num_train + num_val]
x_test, y_test = input_x[-num_test:], y[-num_test:]

train_dataloader = construct_dataloader(x_train,y_train,'train')
val_dataloader = construct_dataloader(x_val,y_val,'valid')
test_dataloader = construct_dataloader(x_test,y_test,'valid')

In [25]:
all_dataloader =  DataLoader(dataset=input_x, batch_size=BATCH_SIZE, shuffle=False)

In [26]:
print(train_dataloader.dataset.__len__())
print(val_dataloader.dataset.__len__())
print(test_dataloader.dataset.__len__())

23981
3426
6852


### 2.2模型

In [27]:
class LSTM_layer(torch.nn.Module):
    def __init__(self,output_size):
        super(LSTM_layer, self).__init__()
        batch_first=True
        hidden_size=64
        self.lstm1 = torch.nn.LSTM(num_layers = 1,batch_first=batch_first,
                               input_size=1,hidden_size = hidden_size)
        self.lstm2 = torch.nn.LSTM(num_layers = 1,batch_first=batch_first,
                               input_size=1,hidden_size = hidden_size)
        self.mlp = Linear(2*hidden_size,output_size)
    def forward(self, x):
        #x: batch numnodes seqlen
        x=x.reshape(x.shape[0]*x.shape[1],x.shape[2],1)
        out1,(hn,cn) = self.lstm1(x[:,:TIME_LEN])
        out2,(hn,cn) = self.lstm2(x[:,TIME_LEN:])# batchsize*n timestep hidden_size
        out1 = out1.relu()
        out2 = out2.relu()#[:,-1,:]
        out = self.mlp(torch.cat((out1,out2),dim=-1))
        out.relu()
        return out.transpose(1,2) #->batch*n hiddensize(channel) timestep 
    
class ResNet_Block(torch.nn.Module):
    def __init__(self,input_channels,output_channels,ks):
        super(ResNet_Block, self).__init__()
        self.align=torch.nn.Conv1d(in_channels=input_channels,out_channels=output_channels,kernel_size=ks)
        self.conv1=torch.nn.Conv1d(in_channels=input_channels,out_channels=output_channels,kernel_size=ks)
        self.bn1 = nn.BatchNorm2d(output_channels)
        self.conv2=torch.nn.Conv1d(in_channels=output_channels,out_channels=output_channels,kernel_size=3,padding=1)
        self.bn2 = nn.BatchNorm2d(output_channels)
    def forward(self,x):# x: batch channel height
        x_align=self.align(x)
        out=self.conv1(x)
        #out=self.bn1(out)
        out=out.relu()
        out=self.conv2(out)
        #out=self.bn2(out)
        #print(out.shape,x.shape)
        out=out+x_align
        return out.relu()

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.lstm = LSTM_layer(output_size=64)
        res_list=[]
        for i in range(3):
            res_list.append(ResNet_Block(input_channels=64,output_channels=64,ks=4))
        res_list.append(ResNet_Block(input_channels=64,output_channels=64,ks=3)) #3*3+2
        self.res = nn.Sequential(*res_list)
        self.mlp = Linear(64,3)
    def forward(self, x):
        #x: batch numnodes seqlen
        out = self.lstm(x)
        out = self.res(out).squeeze(-1)
        out = self.mlp(out)
        return out

In [28]:
model=Model()
print(model)
optimizer =  torch.optim.Adam(model.parameters(), lr=3e-3, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-3)

Model(
  (lstm): LSTM_layer(
    (lstm1): LSTM(1, 64, batch_first=True)
    (lstm2): LSTM(1, 64, batch_first=True)
    (mlp): Linear(in_features=128, out_features=64, bias=True)
  )
  (res): Sequential(
    (0): ResNet_Block(
      (align): Conv1d(64, 64, kernel_size=(4,), stride=(1,))
      (conv1): Conv1d(64, 64, kernel_size=(4,), stride=(1,))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResNet_Block(
      (align): Conv1d(64, 64, kernel_size=(4,), stride=(1,))
      (conv1): Conv1d(64, 64, kernel_size=(4,), stride=(1,))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

### 2.3训练

In [29]:
device = "cuda:0"
model = model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()

In [30]:
def get_acc(out,y):
    predict = torch.argmax(out, dim=-1)
    acc_number = (predict == y).sum()
    return acc_number

In [31]:
def train(optimizer,dataloader):
    acc = 0
    number = 0
    total_loss = 0
    total_batch = 0
    for batch in tqdm.tqdm(dataloader):
        x,y=batch
        x=x.to(device)
        y=y.to(device)
        out = model(x)
        loss = loss_fn(out,y.reshape(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        acc+=get_acc(out,y.reshape(-1))
        number+=len(out)
        total_loss+=loss.detach()
        total_batch+=1
    return acc/number,total_loss/total_batch

In [32]:
def evaluate(dataloader):
    with torch.no_grad():
        acc = 0
        number = 0
        total_loss = 0
        total_batch = 0
        y_hat = []
        y_real = []
        for batch in tqdm.tqdm(dataloader):
            x,y=batch
            x=x.to(device)
            y=y.to(device)
            out = model(x)
            loss = loss_fn(out,y.reshape(-1))
            acc += get_acc(out,y.reshape(-1))
            number+=len(out)
            total_loss+=loss
            total_batch+=1
            
            predict = torch.argmax(out, dim=-1)
            y_hat.append(predict.reshape(-1))
            y_real.append(y.reshape(-1))
        y_hat = torch.cat(y_hat,dim=-1).cpu().numpy()
        y_real = torch.cat(y_real,dim=-1).cpu().numpy()
        print(y_hat.shape,y_real.shape)
        print(y_hat.mean(),y_hat.std())
        print(y_real.mean(),y_real.std())
        print('皮尔逊系数 = ',pearsonr(y_hat,y_real))
    return acc/number,total_loss/total_batch

In [33]:
EPOCH = 10
for epoch in range(EPOCH):
    train_acc,train_loss = train(optimizer,train_dataloader)
    val_acc,val_loss = evaluate(val_dataloader)
    log = '{}-th epoch | trian: acc={} loss={} | val:  acc={} loss={}'
    print(log.format(epoch,train_acc,train_loss,val_acc,val_loss))

100%|███████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 59.07it/s]
/home/hqh/anaconda3/envs/torch1.8_second/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


(709182,) (709182,)
1.0 0.0
0.8212207867655975 0.6975936393948747
皮尔逊系数 =  (nan, nan)
0-th epoch | trian: acc=0.4790164828300476 loss=0.996900737285614 | val:  acc=0.48140108585357666 loss=0.9691302180290222


100%|███████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 59.82it/s]


(709182,) (709182,)
0.9999168055590807 0.009120719248187425
0.8212207867655975 0.6975936393948747
皮尔逊系数 =  (0.004310975423586499, 0.0002829715788930834)
1-th epoch | trian: acc=0.4865548610687256 loss=0.9688684940338135 | val:  acc=0.48142364621162415 loss=0.9555522799491882


100%|███████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 59.52it/s]


...

(709182,) (709182,)
0.6931521104596563 0.5945535452903948
0.8212207867655975 0.6975936393948747
皮尔逊系数 =  (0.32731800928058513, 0.0)
7-th epoch | trian: acc=0.6323778033256531 loss=0.8060788512229919 | val:  acc=0.642961859703064 loss=0.7845543026924133


100%|███████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 59.71it/s]


(709182,) (709182,)
0.7157499767337581 0.5922068288334794
0.8212207867655975 0.6975936393948747
皮尔逊系数 =  (0.32916839956440364, 0.0)
8-th epoch | trian: acc=0.633335292339325 loss=0.8040560483932495 | val:  acc=0.6438079476356506 loss=0.7816556096076965


100%|███████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 59.60it/s]

(709182,) (709182,)
0.7189564878973239 0.6065893106140313
0.8212207867655975 0.6975936393948747
皮尔逊系数 =  (0.33214217558429093, 0.0)
9-th epoch | trian: acc=0.6332198977470398 loss=0.8037219643592834 | val:  acc=0.6431183815002441 loss=0.7830707430839539


In [34]:
torch.save(model,'./model/pre.th')

In [35]:
n=(sig.shape[0]-1)*sig.shape[1]
print((sig[:-1]==sig[1:]).sum()/n)

0.37001546401198776


In [36]:
def get_sign(data_loader):
    sign = []
    with torch.no_grad():
        for batch in tqdm.tqdm(data_loader):
            x=batch
            x=x.to(device)
            out = model(x)
            predict = torch.argmax(out, dim=-1)
            sign.append(predict.reshape(x.shape[0],x.shape[1]))
    sign=torch.cat(sign,dim=0).float()
#     sign=torch.where(sign==0,0.5,sign)
#     sign=torch.where(sign==2,-1,sign)
    print(sign.shape)
    return sign

In [37]:
SIGN = get_sign(all_dataloader)

100%|█████████████████████████████████████████████████████████████████████████████████| 268/268 [00:04<00:00, 56.38it/s]

torch.Size([34259, 207])


In [38]:
print(SIGN[50][20:99])

tensor([0., 2., 2., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 2., 1., 1., 1.,
        1., 2., 2., 0., 0., 0., 0., 2., 1., 2., 1., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 2., 1., 2., 1., 1., 2., 0., 0., 1., 1., 2., 2., 2., 1., 1., 1.,
        1., 2., 2., 0., 1., 0., 2., 2., 0., 0., 1., 0., 1., 1., 0., 0., 1., 2.,
        1., 1., 1., 2., 0., 2., 0.], device='cuda:0')


In [39]:
print(torch.where(SIGN==0)[0].shape)
print(SIGN[torch.where(SIGN==0)])
SIGN[torch.where(SIGN==0)]=0.5
SIGN[torch.where(SIGN==2)]=-1

torch.Size([2391368])
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')


In [40]:
print(SIGN[50][20:99])
print(SIGN.shape)

tensor([ 0.5000, -1.0000, -1.0000,  1.0000,  1.0000,  0.5000,  0.5000,  1.0000,
         0.5000,  1.0000,  0.5000,  0.5000,  0.5000,  1.0000, -1.0000,  1.0000,
         1.0000,  1.0000,  1.0000, -1.0000, -1.0000,  0.5000,  0.5000,  0.5000,
         0.5000, -1.0000,  1.0000, -1.0000,  1.0000,  0.5000,  1.0000,  1.0000,
         0.5000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000,
        -1.0000,  1.0000,  1.0000, -1.0000,  0.5000,  0.5000,  1.0000,  1.0000,
        -1.0000, -1.0000, -1.0000,  1.0000,  1.0000,  1.0000,  1.0000, -1.0000,
        -1.0000,  0.5000,  1.0000,  0.5000, -1.0000, -1.0000,  0.5000,  0.5000,
         1.0000,  0.5000,  1.0000,  1.0000,  0.5000,  0.5000,  1.0000, -1.0000,
         1.0000,  1.0000,  1.0000, -1.0000,  0.5000, -1.0000,  0.5000],
       device='cuda:0')
torch.Size([34259, 207])


## 计算最终的M

In [41]:
edge_start_sign=SIGN[:,edge_index[0]].to('cpu')
edge_c=torch.FloatTensor(DIFF[TIME_LEN+1:])
print(edge_start_sign.shape)
print(edge_c.shape)

torch.Size([34259, 1722])
torch.Size([34259, 1722])


In [42]:
M=edge_start_sign/edge_c #改成除法看看效果

In [43]:
print(M[0])

tensor([ 2., -1., -1.,  ...,  1.,  1., -2.])


In [44]:
M_pos=M.clone()
M_pos[torch.where(M<0)]=0
print(M_pos[0])

tensor([2., 0., 0.,  ..., 1., 1., 0.])


In [45]:
M_neg=M.clone()
M_neg[torch.where(M>0)]=0
#x
M_neg=-M_neg#尴尬了一开始没加这个
M_neg[torch.where(M_neg<=0)]=0
print(M_neg[0])

tensor([0., 1., 1.,  ..., 0., 0., 2.])


In [46]:
print(sys.getsizeof(M) / 1024 , 'kB')
print(M_neg.shape)

0.0625 kB
torch.Size([34259, 1722])


In [47]:
# torch.save(M_pos,'/home/hqh/DataSetFile/metrla/M_pos.pt')

In [48]:
# torch.save(M_neg,'/home/hqh/DataSetFile/metrla/M_neg.pt')

In [49]:
print(M)

tensor([[ 2.0000, -1.0000, -1.0000,  ...,  1.0000,  1.0000, -2.0000],
        [ 2.0000,  1.0000,  2.0000,  ..., -0.5000, -0.5000,  1.0000],
        [ 2.0000,  2.0000,  1.0000,  ...,  2.0000,  2.0000,  2.0000],
        ...,
        [ 2.0000,  1.0000,  1.0000,  ..., -0.5000, -0.5000,  1.0000],
        [-2.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000,  2.0000],
        [ 1.0000,  1.0000,  1.0000,  ...,  1.0000, -0.5000,  1.0000]])


# 计算ND_h sig ND_T

#### ND_sig

In [50]:
def get_NDsig(data_loader):
    sign = []
    with torch.no_grad():
        for batch in tqdm.tqdm(data_loader):
            x=batch
            x=x.to(device)
            out = model(x).softmax(dim=-1).cpu()
            sign.append(out.reshape(x.shape[0],x.shape[1],3))
    sign=torch.cat(sign,dim=0).float()
    print(sign.shape)
    return sign

In [51]:
ND_sig=get_NDsig(all_dataloader)

100%|█████████████████████████████████████████████████████████████████████████████████| 268/268 [00:05<00:00, 51.92it/s]


torch.Size([34259, 207, 3])


In [52]:
print(ND_sig[0][:])

tensor([[1.6379e-01, 7.0144e-01, 1.3478e-01],
        [4.5572e-01, 2.0645e-01, 3.3783e-01],
        [5.5948e-02, 8.5607e-01, 8.7987e-02],
        [3.9971e-01, 2.5025e-01, 3.5004e-01],
        [5.0247e-01, 2.5922e-01, 2.3831e-01],
        [6.3247e-01, 1.3343e-01, 2.3410e-01],
        [3.4690e-01, 5.1856e-01, 1.3454e-01],
        [2.9286e-01, 4.4679e-02, 6.6246e-01],
        [1.6743e-01, 4.6367e-01, 3.6890e-01],
        [2.6607e-01, 1.3483e-01, 5.9910e-01],
        [3.0172e-01, 4.8378e-01, 2.1451e-01],
        [6.1474e-01, 9.2751e-02, 2.9251e-01],
        [3.8399e-01, 1.0411e-01, 5.1189e-01],
        [4.2671e-01, 3.9013e-01, 1.8316e-01],
        [2.5180e-01, 2.9837e-01, 4.4983e-01],
        [5.0576e-01, 3.4452e-01, 1.4972e-01],
        [1.5691e-01, 6.0039e-01, 2.4270e-01],
        [3.3980e-01, 1.1243e-01, 5.4777e-01],
        [2.0924e-01, 4.4655e-01, 3.4421e-01],
        [2.4599e-01, 5.9985e-01, 1.5416e-01],
        [5.6565e-01, 5.9508e-02, 3.7484e-01],
        [1.5042e-01, 2.3513e-01, 6

#### ND_h

In [53]:
start_time = datetime.datetime.now()
DIFF = data[:,edge_index[0]]-data[:,edge_index[1]]
print(datetime.datetime.now()-start_time)
DIFF = (DIFF+1)/(data[:,edge_index[0]]+1)
print(datetime.datetime.now()-start_time)
DIFF_ABS = np.absolute(DIFF)
print(datetime.datetime.now()-start_time)

0:00:00.549237
0:00:01.144302
0:00:01.320636


In [54]:
eplision=0.02
print(sys.getsizeof(DIFF) / 1024 / 1024/1024, 'GB')
print(DIFF.shape)
number = DIFF.shape[0]*DIFF.shape[1]
# print( len(DIFF[np.isnan(DIFF)])/number)
# print( len(DIFF[np.isinf(DIFF)])/number)
print(DIFF_ABS.mean(),DIFF_ABS.std())
print('同配比例：',((DIFF_ABS<=eplision).sum())/number)
print('异配正比例：',((DIFF>eplision).sum())/number)
print('异配负比例：',((DIFF<-eplision).sum())/number)

0.4397064447402954 GB
(34272, 1722)
0.9486729411882304 6.329240266700626
同配比例： 0.2273149097037189
异配正比例： 0.4731735173744294
异配负比例： 0.29951157292185165


In [55]:
DIFF[DIFF>eplision]=1
DIFF[DIFF_ABS<=eplision]=0
DIFF[DIFF<-eplision]=2

In [56]:
DIFF[0]

array([0., 2., 1., ..., 0., 2., 0.])

In [57]:
edge_spatial_types=torch.LongTensor(DIFF[TIME_LEN+1:,])

In [58]:
edge_spatial_types.shape

torch.Size([34259, 1722])

In [59]:
edge_spatial_types[-1][1:252]

tensor([0, 0, 2, 1, 2, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        0, 1, 1, 0, 0, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2,
        2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 1,
        1, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 0, 2, 1, 0, 1, 1, 2, 2, 0, 1, 2, 0,
        2, 2, 0, 2, 1, 1, 1, 2, 2, 2, 0, 2, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 1, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 1])

In [60]:
ND_h=[]
for edge_type in tqdm.tqdm(edge_spatial_types):
    ND_h_=torch.zeros(ND_sig.shape[1],3)
    for i in range(len(edge_type)):#只能串行计算
        ND_h_[edge_index[0][i],edge_type[i]]+=1
    ND_h.append(ND_h_)

100%|█████████████████████████████████████████████████████████████████████████████| 34259/34259 [23:23<00:00, 24.41it/s]


In [61]:
ND_h_torch=torch.stack(ND_h,dim=0)
print(ND_h_torch.shape)
# print(ND_h_torch[2][:])

torch.Size([34259, 207, 3])


### ND_t

In [62]:
degree = ND_h_torch.sum(dim=-1).unsqueeze(-1).repeat_interleave(3,dim=-1)
print(degree.shape)

torch.Size([34259, 207, 3])


In [63]:
ND_t = ND_h_torch*ND_sig/degree/degree

In [64]:
print(ND_t[0][:])

tensor([[1.1374e-03, 9.7422e-03, 8.4236e-03],
        [7.1206e-03, 3.2258e-03, 3.1672e-02],
        [1.3814e-03, 4.2275e-02, 3.2588e-03],
        [4.0787e-03, 1.2768e-03, 1.9645e-02],
        [1.9628e-02, 6.0754e-03, 0.0000e+00],
        [3.7424e-03, 2.3685e-03, 1.2467e-02],
        [2.1239e-02, 4.2331e-02, 0.0000e+00],
        [7.4708e-03, 9.1181e-04, 1.6900e-02],
        [3.4170e-03, 4.7313e-02, 7.5286e-03],
        [1.0643e-02, 2.1572e-02, 0.0000e+00],
        [2.4137e-02, 3.8702e-02, 8.5803e-03],
        [1.5179e-02, 8.0155e-03, 0.0000e+00],
        [2.1551e-02, 1.0624e-03, 2.6117e-03],
        [4.2671e-03, 3.5112e-02, 0.0000e+00],
        [1.5737e-02, 4.6620e-03, 2.1086e-02],
        [2.4976e-02, 2.1267e-02, 0.0000e+00],
        [0.0000e+00, 1.6631e-03, 1.2101e-02],
        [4.0213e-03, 0.0000e+00, 3.5654e-02],
        [2.3249e-02, 4.9616e-02, 3.8246e-02],
        [6.0989e-03, 3.4702e-02, 1.2741e-03],
        [1.5712e-02, 2.0662e-03, 7.8092e-03],
        [0.0000e+00, 4.4087e-02, 3

In [65]:
torch.save(ND_t,'/home/hqh/DataSetFile/metrla/ND_t.pt')